In [5]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType, LongType
import pyspark.sql.functions as psf
from Buoi_4_ETL import ETL_daily

In [6]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()
read_path = r'C:\\Users\\ADMIN\\PycharmProjects\\Bigdata-1\\Data\\Dataset\\log_content\\'
save_path = r"C:\\Users\\ADMIN\\PycharmProjects\\Bigdata-1\\Data\Dataset\\Clean_data\\"
file_name = '20220401'
date = '2022-04-1'

In [7]:
emptyRDD = spark.sparkContext.emptyRDD()

schema = StructType([
    StructField('Contract', StringType(), True), 
    StructField('Type', StringType(), False), 
    StructField('Giải Trí', LongType(), True), 
    StructField('Phim Truyện', LongType(), True), 
    StructField('Thiếu Nhi', LongType(), True), 
    StructField('Thể Thao', LongType(), True), 
    StructField('Truyền Hình', LongType(), True), 
    StructField('Date', StringType(), False)])

monthly_df = spark.createDataFrame(emptyRDD,schema)
monthly_df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Type: string (nullable = false)
 |-- Giải Trí: long (nullable = true)
 |-- Phim Truyện: long (nullable = true)
 |-- Thiếu Nhi: long (nullable = true)
 |-- Thể Thao: long (nullable = true)
 |-- Truyền Hình: long (nullable = true)
 |-- Date: string (nullable = false)



In [8]:
import pandas as pd
start_date, end_date = "2022-04-01", "2022-04-30"

day_range = pd.period_range(start=start_date, end=end_date, freq='D')

day_list = [day.strftime("%Y-%m-%d") for day in day_range]
filename_list = [day.strftime("%Y%m%d") for day in day_range]

for i in range(len(day_list)):
    date = day_list[i]
    file_name = filename_list[i]

    ETL = ETL_daily(spark)
    ETL.load_data(read_path, file_name)
    merge_df = ETL.transform_data(date)
    monthly_df = monthly_df.union(merge_df)
    # ETL.save_data(save_path, file_name)

------------------------------
Read data from HDFS
------------------------------
Transforming data
------------------------------
------------------------------
Pivoting data
------------------------------
Read data from HDFS
------------------------------
Transforming data
------------------------------
------------------------------
Pivoting data
------------------------------
Read data from HDFS
------------------------------
Transforming data
------------------------------
------------------------------
Pivoting data
------------------------------
Read data from HDFS
------------------------------
Transforming data
------------------------------
------------------------------
Pivoting data
------------------------------
Read data from HDFS
------------------------------
Transforming data
------------------------------
------------------------------
Pivoting data
------------------------------
Read data from HDFS
------------------------------
Transforming data
--------------------

In [9]:
monthly_df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Type: string (nullable = false)
 |-- Giải Trí: long (nullable = true)
 |-- Phim Truyện: long (nullable = true)
 |-- Thiếu Nhi: long (nullable = true)
 |-- Thể Thao: long (nullable = true)
 |-- Truyền Hình: long (nullable = true)
 |-- Date: string (nullable = false)



In [10]:
monthly_df.groupBy('Date').count().show()

+----------+-------+
|      Date|  count|
+----------+-------+
|2022-04-01|1483420|
|2022-04-02|1470096|
|2022-04-03|1448363|
|2022-04-04|1448944|
|2022-04-05|1442091|
|2022-04-06|1443856|
|2022-04-07|1443485|
|2022-04-08|1440827|
|2022-04-09|1444540|
|2022-04-10|1407723|
|2022-04-11|1464487|
|2022-04-12|1430434|
|2022-04-13|1448919|
|2022-04-14|1438174|
|2022-04-15|1447543|
|2022-04-16|1493387|
|2022-04-17|1467775|
|2022-04-18|1431460|
|2022-04-19|1449971|
|2022-04-20|1434339|
+----------+-------+
only showing top 20 rows



In [11]:
month_summary = monthly_df.groupBy('Contract').agg(sum('Giải Trí').alias('Giai_Tri'),
                                   sum('Phim Truyện').alias('Phim_Truyen'),
                                   sum('Thiếu Nhi').alias('Thieu_Nhi'),
                                   sum('Thể Thao').alias('The_Thao'),
                                   sum('Truyền Hình').alias('Truyen_Hinh'))

In [12]:
month_summary.show(10)

+---------+-----------------+--------------------+------------------+-----------------+--------------------+----------+
| Contract|Giai_Tri_Duration|Phim_Truyen_Duration|Thieu_Nhi_Duration|The_Thao_Duration|Truyen_Hinh_Duration|most_match|
+---------+-----------------+--------------------+------------------+-----------------+--------------------+----------+
|HTFD11598|             null|               15551|              null|             null|               42919|   42919.0|
|HPFD48556|               69|                null|              null|             null|             1468328| 1468328.0|
|NBFD10014|             null|                null|              null|             null|             1596494| 1596494.0|
|HNH619088|             null|               77275|             11361|             null|              917930|  917930.0|
|HNH036174|             null|               62674|              null|             null|              354879|  354879.0|
|DNH067877|             null|           

In [12]:
minf = lit(float("-inf"))
row_max = greatest(*[coalesce(col(x), minf) for x in month_summary.columns[1:6]])
cond = "when" + ".when".join(["(col('" + c + "') == col('max_value'), lit('" + c + "'))" for c in month_summary.columns[1:6]])

month_summary = month_summary.withColumn("max_value", row_max).withColumn("most_watch", eval(cond))

In [14]:
month_summary = month_summary.withColumn("taste", array(*[when(col(c).isNull(), lit(c)) for c in month_summary.columns[1:6]])).withColumn("taste", expr("filter(taste, x -> x is not null)"))